<a href="https://colab.research.google.com/github/popescuaaa/playground/blob/master/Gans_and_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Create a simple neural network to classify iris plants

## Setup

In [106]:
import torch
import yaml
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np

## Custom dataset
## Must povide data + labels
General structure for data:
[f, f, f, f, f, l] f = fature, l = label

In [107]:
LABELS = ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']

In [108]:
class IrisDataset(Dataset):
    def __init__(self):
        self.raw_data = np.loadtxt('./Iris.csv', delimiter=',', dtype=np.str)
        self.len = self.raw_data.shape[0] - 1
        self.relevant_data = [[float(item[1]), float(item[2]), float(item[3]), float(item[4])] \
                     for item in self.raw_data[1:len(self.raw_data)]]

        # Processing data
        self.labels = [ [float(LABELS.index(item[5]))] for item in self.raw_data[1:len(self.raw_data)] ]

        self.data = [(None, None)] * len(self.relevant_data)

        for idx in range(len(self.relevant_data)):
          self.data[idx] = ( torch.Tensor(self.relevant_data[idx]), torch.Tensor(self.labels[idx]))

    def get_features_list(self):
      return self.raw_data[0][1:len(self.raw_data[0]) - 1]

    def __getitem__(self, index):
        data, label = self.data[index] # (data, label)
        return data, label

    def __len__(self):
        return self.len

In [109]:
ds = IrisDataset()

## Dataloader
Dataset + sampler

In [110]:
 # load yaml data
with open('simple_network.yaml', 'r') as f:
    config = yaml.load(f)

train_ds, test_ds = IrisDataset(), IrisDataset()

train_loader = DataLoader(dataset=train_ds, batch_size=int(config['batch_size']), shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=int(config['batch_size']), shuffle=True)


## NN

In [111]:
class IrisNet(nn.Module):
    def __init__(self, cfg):
        super(IrisNet, self).__init__()
        self.input_size = cfg['input_size']
        self.hidden_layer_1 = cfg['hidden_layer_1']
        self.hidden_layer_2 = cfg['hidden_layer_2']
        self.num_classes = cfg['num_classes']

        # define network structure
        self.fc1 = nn.Linear(self.input_size, self.hidden_layer_1)
        self.activation1 = nn.ReLU()

        self.fc2 = nn.Linear(self.hidden_layer_1, self.hidden_layer_2)
        self.activation2 = nn.ReLU()

        self.fc3 = nn.Linear(self.hidden_layer_2, self.num_classes)


    def forward(self, batch):
        out = self.fc1(batch)
        out = self.activation1(out)
        out = self.fc2(out)
        out = self.activation2(out)
        out = self.fc3(out)
        return out



## Criterion and Optimizer

In [112]:
iris_nn = IrisNet(config)
criterion = nn.MSELoss()
optimizer = optim.SGD(iris_nn.parameters(), lr=0.001, momentum=0.9)

## Training

In [120]:

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = iris_nn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.001
[1,     2] loss: 0.001
[1,     3] loss: 0.001
[2,     1] loss: 0.001
[2,     2] loss: 0.001
[2,     3] loss: 0.001
Finished Training


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([60, 1])) that is different to the input size (torch.Size([60, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([30, 1])) that is different to the input size (torch.Size([30, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [121]:
PATH = './iris_net.pth'
torch.save(iris_nn.state_dict(), PATH)

In [122]:
iris_nn_test = IrisNet(config)
iris_nn_test.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [123]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        iris, labels = data
        outputs = iris_nn_test(iris)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test iris data: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test iris data: 1760 %
